# <center> Image Colorization Final

#### Import Libraries

Download data from: https://github.com/VinitaSilaparasetty/Image-Colorization-with-TensorFlow2-and-Keras

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, InputLayer, Conv2D,UpSampling2D,DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape, Flatten,LeakyReLU, Dropout,DepthwiseConv2D
from tensorflow.keras.layers import Flatten,MaxPooling2D,Conv2DTranspose, AveragePooling2D
from tensorflow.keras.applications.vgg16  import VGG16
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from PIL import Image
from tensorflow.keras.utils import plot_model
import cv2


#Load Data

In [ ]:
ab = np.load('ab1.npy')
gray = np.load('gray_scale.npy')

#Prepare Gray Images

In [ ]:
def batch_prep(gray_img, batch_size = 100, prep_in = tf.keras.applications.vgg16.preprocess_input):
    img = np.zeros((batch_size, 224, 224, 3))
    for i in range(0, 3):
        img[:batch_size, :, :,i] = gray_img[:batch_size]
    return prep_in(img)

img_in = batch_prep(gray, batch_size = 300)

#Combine Colorful Images and Gray Images for Input

In [ ]:
def get_rbg(gray_imgs, ab_imgs, n = 10):
    
    #create an empty array to store images
    img1 = np.zeros((n, 224, 224, 3))
    
    img1[:, :, :, 0] = gray_imgs[0:n:]
    img1[:, :, :, 1:] = ab_imgs[0:n:]
    
    #convert all the images to type unit8
    img1 = img1.astype("uint8")
    
    #create a new empty array
    imgs= []
    
    for i in range(0, n):
        imgs.append(cv2.cvtColor(img1[i], cv2.COLOR_LAB2RGB))
        
    #convert the image matrix into a numpy array
    imgs = np.array(imgs)
    
    return imgs

In [ ]:
img_out = tf.keras.applications.vgg16.preprocess_input(get_rbg(gray_imgs = gray, ab_imgs = ab, n = 300))

#Build Model

In [ ]:
model6 = VGG16(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
model = Sequential()
model.add(InputLayer(input_shape=(img_in.shape[1], img_in.shape[2], 3)))
model.add(Model(inputs=model6.inputs, outputs=model6.layers[-10].output))
model.add(UpSampling2D((2, 2)))
model.add(UpSampling2D((2, 2)))
model.add(DepthwiseConv2D(32, (2, 2), activation=tf.nn.relu, padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(DepthwiseConv2D(32, (2, 2), activation=tf.nn.relu, padding='same'))
model.add(layers.ReLU(0.3))
model.add(layers.Dropout(0.4))
model.add(UpSampling2D((2, 2)))
model.add(UpSampling2D((2, 2)))
model.add(DepthwiseConv2D(2, (2, 2), activation=tf.nn.relu, padding='same'))
model.add(layers.ReLU(0.3))
model.add(layers.Dropout(0.2))
model.add(UpSampling2D((2, 2)))
model.add(layers.ReLU(0.3))
model.add(layers.Dropout(0.2))
model.add(AveragePooling2D(pool_size = (2, 2)))
model.add(layers.Dense(units=3))
print(model.summary())

# Set Model Parameters

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),loss = 'mse',metrics=tf.keras.metrics.Accuracy())

#Train Model

In [ ]:
#if you encounter an OOM error, reduce the batch_size to 8
model.fit(img_in, img_out, epochs =5, batch_size = 16)

#Obtain Prediction

In [ ]:
prediction = model.predict(img_in)

#View Prediction

In [ ]:
#display the predicted image
plt.imshow(prediction[29])